# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [2]:
import random, math
import pandas as pd
import time
import os

In [3]:
def save_instance(dataset, instance_name, n, aristas):
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        print(n, len(aristas), file=f) #escribo la cantidad de nodos seguido de la cantidad de aristas
        for arista in aristas: #para todas las aristas
            print(arista[0], file=f, end= " ") #escribo el primer nodo
            print(arista[1], file=f, end= " ") #el segundo
            print(arista[2], file=f, end =" ") # el peso y salto de linea
            
def save_index(dataset, instances):
    with open(F"instancias/{dataset}/instances.txt", "w") as f:
        for instance in instances: 
            print(instance, file=f)

In [3]:
filas_indice = []
for n in range(3, 103):#creo 100 grafos con n=3....103
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    S = []#inicializo la lista donde voy a guardar las aristas
    for i in range(1, n+1):#empiezo en el nodo 1 en vez del nodo 0 por convencion del tp
        for j in range (i+1, n+1):#creo una arista para todo par i, j donde j>i
              S.append([i, j, random.randint(0, 100)]) #la guardo
    save_instance("grafo-propio", F"G-{n}", n, S)
    filas_indice.append(["grafo-propio", F"G-{n}", n, aristas, F"instancias/grafo-propio/G-{n}.txt"])     
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/grafo-propio/indice.csv", index=False, header=True)    

# Peor caso AGM

In [4]:
filas_agm = []
for n in range(3, 103):
    aristas = n*(n-1)/2
    S = []
    for j in range(2,n+1):
        S.append([1, j, 1]) #quiero que el AGM sea una estrella centrada en V1
    for i in range(2, n+1): #el resto
        if i+1<n+1:
            S.append([i,i+1, random.randint(150, 200)]) #la arista hacia el proximo en dfs es MUY costosa
        for j in range(i+2, n+1):
            S.append([i,j, random.randint(20,100)]) #el resto es malo, pero no tanto como las que elije el dfs
    save_instance("peor-caso-agm", F"AGM-{n}",n,S)
    filas_agm.append(["peor-caso-agm",F"AGM-{n}",n,aristas,F"instancias/peor-caso-agm/AGM-{n}.txt"])
pd.DataFrame(filas_agm, columns=["dataset", "instancia", "n", "m", "archivo"]).to_csv("instancias/peor-caso-agm/indice.csv", index=False, header=True)

# Cargamos los grafos para los cuales conocemos los resultados optimos

In [6]:
filas_indice=[]
archivos = os.listdir("instancias/descargas/data")
for instancia in archivos:
    n = ""
    for letra in instancia:
        if letra.isnumeric():
            n = n+letra
    n = int(n)
    aristas= n*(n-1)/2 #cantidad de aristas de un grafo Kn
    nombre = instancia.split(".")[0]
    soluciones = open("instancias/descargas/pesos_optimos.txt", "r")
    encontre = False
    peso_opt = 0
    print(nombre)
    while(not encontre):
        linea = soluciones.readline().split(":")
        if nombre == linea[0]:
            encontre = True
            peso_opt = int(linea[1])
        if "FIN" == linea[0]:
            break
    if peso_opt == 0:
            print("falta el optimo del caso " + nombre)
            time.sleep(5)
    filas_indice.append(["sol_optimas", nombre, n, aristas, F"instancias/descargas/data/{nombre}.tsp", peso_opt])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "n", "m", "archivo", "peso_optimo"]).to_csv("instancias/descargas/indice.csv", index=False, header=True)
    


u1432
gr666
rl1304
pr107
rat99
d2103
kroA200
rl1323
p654
u574
pr152
bier127
pcb1173
bays29
u1060
pr76
fl1577
kroB100
u2152
ch150
pr264
gr202
att532
dantzig42
kroB150
eil76
ts225
kroB200
pr144
u724
pa561
linhp318
u2319
pr2392
dsj1000
kroA150
kroA100
burma14
u159
d1655
gr229
gr137
kroC100
ch130
pr136
a280
eil51
lin105
rat195
gil262
d657
bayg29
gr431
fl417
pr299
pr1002
tsp225
nrw1379
ulysses16
vm1748
eil101
vm1084
rl1889
u1817
pr124
berlin52
d1291
d198
pr439
st70
pcb442
gr96
rat783
gr120
pr226
rd400
kroD100
fl1400
ali535
lin318
rat575
kroE100
att48
ulysses22
d493
rd100


In [ ]:
#agrego comentario para pushear
